In [129]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
import re
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [130]:
area = "bangkok"
data = pd.read_csv("../data/airbnb_" + area + "_cleaned.csv", index_col=0)

In [131]:
data["property_type"].value_counts().head(10)

Entire rental unit                     4558
Entire condominium (condo)             3492
Private room in rental unit            1830
Private room in condominium (condo)     899
Private room in residential home        646
Room in boutique hotel                  638
Room in hotel                           528
Private room in townhouse               524
Private room in hostel                  500
Entire serviced apartment               456
Name: property_type, dtype: int64

In [132]:
# keep if property type is Apartment, House or Townhouse
data = data.loc[
    lambda x: x["property_type"].isin(
        [
            "Entire rental unit",
            "Entire condominium (condo)",
            "Entire serviced apartment",
        ]
    )
]

In [133]:
data["f_property_type"] = data["property_type"].astype("category")

In [134]:
data["room_type"].value_counts()

Entire home/apt    8506
Name: room_type, dtype: int64

In [135]:
data.drop("room_type",axis=1,inplace=True)

In [136]:
print(data.columns.tolist())

['scrape_id', 'host_id', 'host_name', 'host_since', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', '

In [137]:
data["neighbourhood_cleansed"].value_counts()

Khlong Toei             1482
Vadhana                 1196
Huai Khwang              867
Ratchathewi              702
Sathon                   439
Bang Rak                 354
Phra Khanong             343
Bang Na                  306
Din Daeng                269
Chatu Chak               260
Parthum Wan              259
Khlong San               220
Phaya Thai               210
Suanluang                202
Bang Sue                 146
Bang Kapi                127
Yan na wa                121
Bang Phlat               119
Thon buri                104
Phasi Charoen             95
Bang Kho laen             80
Lat Krabang               74
Wang Thong Lang           63
Pra Wet                   50
Lak Si                    44
Bang Khen                 37
Bang Khae                 34
Rat Burana                30
Chom Thong                30
Phra Nakhon               28
Bangkok Yai               27
Pom Prap Sattru Phai      26
Bangkok Noi               22
Don Mueang                20
Bueng Kum     

In [138]:
# neighbourhood_cleansed as factors

data["f_neighbourhood_cleansed"] = data["neighbourhood_cleansed"].astype("category")

---------

### Create Numerical variables

In [139]:
data["usd_price_day"] = data["price"]
data["p_host_response_rate"] = data["host_response_rate"].fillna(0).astype(int)
# rename cleaning_fee column

data = data.rename(columns={"cleaning_fee":"usd_cleaning_fee"})

In [140]:
data[
    [
        "accommodates",
        "bathrooms",
        "review_scores_rating",
        "number_of_reviews",
        "reviews_per_month",
        "minimum_nights",
        "beds",
    ]
].isnull().sum()

accommodates               0
bathrooms               8506
review_scores_rating    2681
number_of_reviews          0
reviews_per_month       2681
minimum_nights             0
beds                     139
dtype: int64

In [141]:
# add new numeric columns from certain columns

numericals = [
    "accommodates",
    "review_scores_rating",
    "number_of_reviews",
    "reviews_per_month",
    "minimum_nights",
    "beds",
]

for col in numericals:
    data["n_" + col] = pd.to_numeric(data[col], errors="coerce")

In [142]:
# create days since first review

data["n_days_since"] = (
    data.calendar_last_scraped.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    - data.first_review.fillna("1950-01-01").apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
).dt.days

data["n_days_since"] = np.where(data.first_review.isnull(), np.nan, data.n_days_since)

In [143]:
amenities_to_keep = pd.read_pickle("amenities_to_keep.pkl")

In [144]:
# create dummy vars
dummies = amenities_to_keep.index.tolist()

for col in dummies:
    data["d_" + (re.sub("/|\s|-", "", col)).replace("(s)", "s").lower()] = data[col]

In [145]:
# keep columns if contain d_, n_,f_, p_, usd_ and some others

data = pd.concat(
    [
        data.filter(
            regex="^d_.*|^n_.*|^f_.*|^p_.*|^usd_.*",
        ),
        data[
            [
                "price",
                "neighbourhood_cleansed",
                "property_type",
            ]
        ],
    ],
    axis=1,
)

In [146]:
 data["price"].astype(float)

id
27934         NaN
28354         NaN
36458       567.0
59221       910.0
61000       820.0
            ...  
51813097    880.0
51851215    960.0
51851676      NaN
51854076      NaN
51854776      NaN
Name: price, Length: 8506, dtype: float64

In [147]:
# Squares and further values to create
data = data.assign(
    n_accommodates2=lambda x: x["n_accommodates"] ** 2,
    ln_accommodates=lambda x: np.log(x["n_accommodates"]),
    ln_accommodates2=lambda x: np.log(x["n_accommodates"]) ** 2,
    ln_beds=lambda x: np.log(x["n_beds"]),
    ln_number_of_reviews=lambda x: np.log(x["n_number_of_reviews"] + 1),
)

In [148]:
data.columns

Index(['f_property_type', 'f_neighbourhood_cleansed', 'usd_price_day',
       'p_host_response_rate', 'n_accommodates', 'n_review_scores_rating',
       'n_number_of_reviews', 'n_reviews_per_month', 'n_minimum_nights',
       'n_beds', 'n_days_since', 'd_airconditioning', 'd_longtermstaysallowed',
       'd_essentials', 'd_wifi', 'd_hangers', 'd_shampoo', 'd_hairdryer',
       'd_washer', 'd_dedicatedworkspace', 'd_kitchen', 'd_smokealarm', 'd_tv',
       'd_elevator', 'd_hotwater', 'd_freeparkingonpremises',
       'd_fireextinguisher', 'd_pool', 'd_gym', 'd_refrigerator', 'd_iron',
       'd_microwave', 'd_dishesandsilverware', 'd_dryer',
       'd_carbonmonoxidealarm', 'd_bedlinens', 'd_privateentrance',
       'd_firstaidkit', 'd_cabletv', 'd_tvwithstandardcable',
       'd_patioorbalcony', 'd_lockonbedroomdoor', 'd_cookingbasics',
       'd_luggagedropoffallowed', 'd_stove', 'd_heating',
       'd_extrapillowsandblankets', 'd_breakfast', 'd_hostgreetsyou',
       'd_backyard', 'd_

In [149]:
# Pool accomodations with 0,1,2,10 bathrooms

bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 10)], closed="left")
f_bath = pd.cut(data["n_bathrooms"].to_list(), bins, labels=["0", "1", "2"])
f_bath.categories = [0, 1, 2]
data["f_bathroom"] = f_bath

f_bath.value_counts(dropna=False)

KeyError: 'n_bathrooms'

In [152]:
# Pool num of reviews to 3 categories: none, 1-51 and >51

bins = pd.IntervalIndex.from_tuples(
    [(0, 1), (1, 51), (51, max(data.n_number_of_reviews))], closed="left"
)
fnor = pd.cut(data["n_number_of_reviews"].to_list(), bins, labels=["0", "1", "2"])
fnor.categories = [0, 1, 2]
data["f_number_of_reviews"] = fnor
data["f_number_of_reviews"].value_counts(dropna=False)

1      4839
0      2681
2       985
NaN       1
Name: f_number_of_reviews, dtype: int64

In [153]:
# Pool and categorize the number of minimum nights: 1,2,3, 3+

bins = pd.IntervalIndex.from_tuples(
    [(1, 2), (2, 3), (3, max(data.n_minimum_nights))], closed="left"
)
f_min_n = pd.cut(data["n_minimum_nights"].to_list(), bins, labels=["1", "2", "3"])
f_min_n.categories = [1, 2, 3]
data["f_minimum_nights"] = f_min_n
data["f_minimum_nights"].value_counts(dropna=False)

3      3848
1      3697
2       960
NaN       1
Name: f_minimum_nights, dtype: int64

In [155]:
# Change Infinite values with NaNs
data = data.replace([np.inf, -np.inf], np.nan)

In [156]:
# ------------------------------------------------------------------------------------------------
# where do we have missing variables now?
to_filter = data.isna().sum()
to_filter[to_filter > 0]

usd_price_day             4832
n_review_scores_rating    2681
n_reviews_per_month       2681
n_beds                     139
n_days_since              2681
price                     4832
ln_beds                    433
f_minimum_nights             1
f_number_of_reviews          1
dtype: int64

In [157]:
# what to do with missing values?
# 1. drop if no target
data = data.loc[lambda x: x.price.notnull()]

In [158]:
# 2. imput when few, not that important
data = data.assign(
    n_beds=lambda x: np.where(x["n_beds"].isnull(), x["n_accommodates"], x["n_beds"]),
    f_minimum_nights=lambda x: x["f_minimum_nights"].fillna(1),
    f_number_of_reviews=lambda x: x["f_number_of_reviews"].fillna(1),
    ln_beds=lambda x: x["ln_beds"].fillna(0),
)

In [160]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

n_review_scores_rating    905
n_reviews_per_month       905
n_days_since              905
dtype: int64

In [161]:
# 4. Replace missing variables re reviews with zero, when no review + add flags
data = data.assign(
    flag_days_since=np.multiply(data.n_days_since.isna(), 1),
    n_days_since=data.n_days_since.fillna(np.median(data.n_days_since.dropna())),
    flag_review_scores_rating=np.multiply(data.n_review_scores_rating.isna(), 1),
    n_review_scores_rating=data.n_review_scores_rating.fillna(
        np.median(data.n_review_scores_rating.dropna())
    ),
    flag_reviews_per_month=np.multiply(data.n_reviews_per_month.isna(), 1),
    n_reviews_per_month=data.n_reviews_per_month.fillna(
        np.median(data.n_reviews_per_month.dropna())
    ),
    flag_n_number_of_reviews=np.multiply(data.n_number_of_reviews.isna(), 1),
)

In [162]:
data.flag_days_since.value_counts()

0    2769
1     905
Name: flag_days_since, dtype: int64

In [163]:
# redo features
# Create variables, measuring the time since: squared, cubic, logs
data = data.assign(
    ln_days_since=lambda x: np.log(x["n_days_since"] + 1),
    ln_days_since2=lambda x: np.log(x["n_days_since"] + 1) ** 2,
    ln_days_since3=lambda x: np.log(x["n_days_since"] + 1) ** 3,
    n_days_since2=lambda x: x["n_days_since"] ** 2,
    n_days_since3=lambda x: x["n_days_since"] ** 3,
    ln_review_scores_rating=lambda x: np.log(x["n_review_scores_rating"]),
)

In [164]:
data.ln_days_since = data["ln_days_since"].fillna(0)
data.ln_days_since2 = data["ln_days_since2"].fillna(0)
data.ln_days_since3 = data["ln_days_since3"].fillna(0)

In [165]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

Series([], dtype: int64)

In [166]:
data.describe()

,usd_price_day,p_host_response_rate,n_accommodates,n_review_scores_rating,n_number_of_reviews,n_reviews_per_month,n_minimum_nights,n_beds,n_days_since,d_airconditioning,...,flag_days_since,flag_review_scores_rating,flag_reviews_per_month,flag_n_number_of_reviews,ln_days_since,ln_days_since2,ln_days_since3,n_days_since2,n_days_since3,ln_review_scores_rating
count,3674.000000,3674.0,3674.000000,3674.000000,3674.000000,3674.000000,3674.000000,3674.000000,3674.000000,3674.000000,...,3674.000000,3674.000000,3674.000000,3674.0,3674.000000,3674.000000,3674.000000,3.674000e+03,3.674000e+03,3674.000000
mean,679.878606,0.0,2.590637,4.660400,20.953457,0.666116,18.008710,1.246870,912.750953,0.979314,...,0.246326,0.246326,0.246326,0.0,6.652679,44.685104,302.408656,1.091206e+06,1.660902e+09,-inf
std,194.438174,0.0,0.943304,0.554124,42.359915,0.804098,45.836449,0.650261,508.096341,0.142350,...,0.430929,0.430929,0.430929,0.0,0.653519,7.944816,76.084095,1.370232e+06,3.516874e+09,NaN
min,269.000000,0.0,1.000000,0.000000,0.000000,0.010000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.693147,0.480453,0.333025,1.000000e+00,1.000000e+00,-inf
25%,500.000000,0.0,2.000000,4.650000,1.000000,0.220000,1.000000,1.000000,635.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,6.455199,41.669588,268.985468,4.032250e+05,2.560479e+08,1.536867
50%,699.000000,0.0,2.000000,4.770000,4.000000,0.430000,2.000000,1.000000,790.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,6.673298,44.532906,297.181350,6.241000e+05,4.930390e+08,1.562346
75%,850.000000,0.0,3.000000,4.897500,21.000000,0.780000,28.000000,1.000000,990.000000,1.000000,...,0.000000,0.000000,0.000000,0.0,6.898715,47.592262,328.325431,9.801000e+05,9.702990e+08,1.588724
max,999.000000,0.0,8.000000,5.000000,443.000000,15.370000,730.000000,8.000000,3848.000000,1.000000,...,1.000000,1.000000,1.000000,0.0,8.255569,68.154414,562.653442,1.480710e+07,5.697774e+10,1.609438


In [168]:
data.to_csv("../data/airbnb_bangkok_workfile.csv", index=False)